In [3]:
import pandas as pd
import sqlite3
import os

# === 1. CSV file path ===
csv_file_path = 'account_transactions_enhanced.csv'  # Replace with your actual CSV file name

# === 2. Database path ===
db_path = 'bank_transactions.db'  # Changed to the name used in your app.py for consistency

# === 3. Table name to create in SQLite ===
table_name = 'bank_transactions'  # Changed to match the table name used in your app.py/evaluation_module.py

# === 4. Check if CSV file exists ===
if not os.path.exists(csv_file_path):
    raise FileNotFoundError(f"CSV file not found: {csv_file_path}")

# === 5. Read the CSV into a DataFrame, parsing the 'Date_Timestamp' column as datetime ===
# It's crucial that 'Date_Timestamp' is the correct column name from your CSV.
try:
    df = pd.read_csv(csv_file_path, parse_dates=['Date_Timestamp'])
except KeyError:
    print(f"Warning: 'Date_Timestamp' column not found in CSV. Please ensure the column name is correct.")
    print(f"Available columns: {df.columns.tolist()}")
    # Fallback to reading without parsing dates if column not found, but it will affect datetime handling
    df = pd.read_csv(csv_file_path) # Read again without parsing
except Exception as e:
    print(f"An error occurred while reading the CSV and parsing dates: {e}")
    # Consider more robust error handling or exiting here if date parsing is critical
    df = pd.read_csv(csv_file_path) # Attempt to read without parsing

# Optional: Verify the dtype of the 'Date_Timestamp' column
print(f"DataFrame head:\n{df.head()}")
print(f"\n'Date_Timestamp' column dtype after reading: {df['Date_Timestamp'].dtype}")

# === 6. Connect to SQLite database ===
conn = sqlite3.connect(db_path)

# === 7. Write DataFrame to SQLite table ===
# Pandas will automatically convert datetime objects to ISO 8601 strings when writing to SQL,
# which SQLite stores as TEXT, allowing full date/time functionality in SQL queries.
df.to_sql(table_name, conn, if_exists='replace', index=False)

# === 8. Verify and close ===
print(f"\n✅ Loaded {len(df)} rows into table '{table_name}' in '{db_path}'")

# Optional: Verify data types in the database (querying the schema)
cursor = conn.cursor()
cursor.execute(f"PRAGMA table_info({table_name});")
db_schema = cursor.fetchall()
print(f"\nDatabase schema for '{table_name}':")
for col_info in db_schema:
    print(f"  Column: {col_info[1]}, Type: {col_info[2]}") # col_info[1] is name, col_info[2] is type

conn.close()
print("\nDatabase connection closed.")

DataFrame head:
       Date_Timestamp   Transaction_ID       Transaction_Details  \
0 2024-01-01 09:35:00  TRNCD7922899379  POS PURCHASE COFFEE SHOP   
1 2024-01-01 09:57:00  TRNCD8593336000   POS PURCHASE RESTAURANT   
2 2024-01-02 09:50:00  TRNQR7788075352  POS PURCHASE COFFEE SHOP   
3 2024-01-02 09:56:00  TRNCD8430233493   POS PURCHASE RESTAURANT   
4 2024-01-03 09:09:00  TRNQR5379445308  POS PURCHASE COFFEE SHOP   

                   Merchant_Counterparty_Name  Withdrawal_Amount  \
0  Yardstick Coffee - Legazpi Village, Makati              308.0   
1           Marugame Udon - SM City Sta. Mesa              768.0   
2          Toby's Estate - Legazpi St, Makati              212.0   
3               Ramen Nagi - Power Plant Mall             1417.0   
4                 Tim Hortons - SM North EDSA              212.0   

   Deposit_Amount  Recurring_Existing_Balance  
0             0.0                     49692.0  
1             0.0                     48924.0  
2             0.0     